In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Dropout
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_dataframe(subset_number):
    turbo_train= pd.read_csv(f"/content/drive/MyDrive/NASAturbofan/train_FD00{subset_number}.txt", sep='\s+',header=None, index_col=False)
    turbo_test = pd.read_csv(f'/content/drive/MyDrive/NASAturbofan/test_FD00{subset_number}.txt', sep='\s+',header=None, index_col=False)
    y_RUL = pd.read_csv(f'/content/drive/MyDrive/NASAturbofan/RUL_FD00{subset_number}.txt', sep='\s+',header=None, index_col=False)
    return turbo_train, turbo_test, y_RUL


In [ ]:
turbo_train_1, turbo_test_1, y_RUL_1 = load_dataframe(1)
turbo_train_2, turbo_test_2, y_RUL_2 = load_dataframe(2)
turbo_train_3, turbo_test_3, y_RUL_3 = load_dataframe(3)
turbo_train_4, turbo_test_4, y_RUL_4 = load_dataframe(4)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/NASAturbofan/train_FD001.txt'

In [ ]:
def add_RUL_column(turbo_train, turbo_test, y_RUL):
  turbo_train.sort_values(by=[0, 1], inplace=True)
  turbo_train['RUL'] = turbo_train.groupby(0).cumcount(ascending=False)

  y_RUL_series=y_RUL[0]
  turbo_test.sort_values(by=[0, 1], inplace=True)
  adjusted_engine_numbers = turbo_test[0].values - 1
  turbo_test['RUL'] = turbo_test.groupby(0).cumcount(ascending=False) + y_RUL_series.iloc[adjusted_engine_numbers].values

  return turbo_train, turbo_test


In [ ]:
turbo_train_1, turbo_test_1=add_RUL_column(turbo_train_1, turbo_test_1, y_RUL_1)
turbo_train_2, turbo_test_2=add_RUL_column(turbo_train_2, turbo_test_2, y_RUL_2)
turbo_train_3, turbo_test_3=add_RUL_column(turbo_train_3, turbo_test_3, y_RUL_3)
turbo_train_4, turbo_test_4=add_RUL_column(turbo_train_4, turbo_test_4, y_RUL_4)

In [ ]:
def complete_dataframe(turbo_train, turbo_test):
  turbofan = pd.concat([turbo_train, turbo_test])
  return turbofan

In [ ]:
turbofan1=complete_dataframe(turbo_train_1, turbo_test_1)
turbofan2=complete_dataframe(turbo_train_2, turbo_test_2)
turbofan3=complete_dataframe(turbo_train_3, turbo_test_3)
turbofan4=complete_dataframe(turbo_train_4, turbo_test_4)

#Preprocessing

##Correlation Analysis

In [ ]:
def compute_pearson(turbofan):
  correlation_matrix = abs(turbofan.corr(method='pearson'))
  target_column = 'RUL'
  correlation = correlation_matrix[target_column]
  scaled_correlation = correlation * 100
  return round(scaled_correlation, 2)

In [ ]:
p1=compute_pearson(turbofan1)
p2=compute_pearson(turbofan2)
p3=compute_pearson(turbofan3)
p4=compute_pearson(turbofan4)

NameError: name 'turbofan1' is not defined

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15, 8), gridspec_kw={'width_ratios': [1, 1, 1, 1.2]})

sns.heatmap(p1.to_frame(), annot=True, center=0, cmap='PiYG', fmt='', ax=axs[0], cbar=False)
axs[0].set_title('Heatmap of FD001')
axs[0].set_ylabel('Correlation Factor')

sns.heatmap(p2.to_frame(), annot=True, center=0, cmap='PiYG', fmt='', ax=axs[1], cbar=False)
axs[1].set_title('Heatmap of FD002')

sns.heatmap(p3.to_frame(), annot=True, center=0, cmap='PiYG', fmt='', ax=axs[2], cbar=False)
axs[2].set_title('Heatmap of FD003')

sns.heatmap(p4.to_frame(), annot=True, center=0, cmap='PiYG', fmt='', ax=axs[3])
axs[3].set_title('Heatmap of FD004')

plt.tight_layout(rect=[0, 0.03, 1, 0.95], pad=2.5)

plt.show()


In [ ]:
def all_correlations(turbofan):
  correlation_matrix = turbofan.corr(method='pearson')
  return round(correlation_matrix, 2)

In [ ]:
pear1=all_correlations(turbofan1)
pear2=all_correlations(turbofan2)
pear3=all_correlations(turbofan3)
pear4=all_correlations(turbofan4)

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(15, 35), gridspec_kw={'height_ratios': [1, 1, 1, 1.2]})

sns.heatmap(pear1, annot=True, center=0, cmap='PiYG', fmt='', ax=axs[0], cbar_kws={"shrink": .5})
axs[0].set_title('Complete Heatmap of FD001')
axs[0].set_xlabel('Correlation Factor')

sns.heatmap(pear2, annot=True, center=0, cmap='PiYG', fmt='', ax=axs[1], cbar_kws={"shrink": .5})
axs[1].set_title('Complete Heatmap of FD002')

sns.heatmap(pear3, annot=True, center=0, cmap='PiYG', fmt='', ax=axs[2], cbar_kws={"shrink": .5})
axs[2].set_title('Complete Heatmap of FD003')

sns.heatmap(pear4, annot=True, center=0, cmap='PiYG', fmt='', ax=axs[3], cbar_kws={"shrink": .5})
axs[3].set_title('Complete Heatmap of FD004')
axs[3].set_xlabel('Correlation Factor')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

In [ ]:
sensor_pairs = []
for i, pear in enumerate([pear1, pear2, pear3, pear4], start=1):
    for sensor1, row in pear.iterrows():
        for sensor2, correlation in row.items():
            if correlation == 1 and sensor1 != sensor2:
                sensor_pairs.append((f'FD00{i}_{sensor1}', f'FD00{i}_{sensor2}'))

sensor_pairs_list= pd.DataFrame(sensor_pairs, columns=['Sensor1', 'Sensor2'])

In [ ]:
sensor_pairs_list

##Exclusions from each dataset based on correlation:
###Subset 1:
 0, 1, 2, 3, 4, 5, 9, 14, 20, 22, 23

###Subset 2:
9, 13, 16, 17, 21, 23, 22,  24, 25

###Subset 3:
0, 1, 2, 3, 4, 5, 9, 19, 20, 22, 23

###Subset 4:
 9, 13, 16, 17, 21, 22, 23, 24, 25

In [ ]:
turbofan1 = turbofan1.drop(columns=[0, 1, 2, 3, 4, 5, 9, 14, 20, 22, 23 ])
turbofan2 = turbofan2.drop(columns=[0, 1, 9, 13, 16, 17, 21, 23, 22, 24, 25])
turbofan3 = turbofan3.drop(columns=[0, 1, 2, 3, 4, 5, 9, 19, 20, 22, 23])
turbofan4 = turbofan4.drop(columns=[0, 1, 9, 13, 16, 17, 21, 22, 23, 24, 25 ])

In [ ]:
turbofan1

##Filtering

In [ ]:
def moving_median( turbofan, window_size, period):
  columns_to_filter = turbofan.columns.drop('RUL')
  filtered_turbo = turbofan.copy()
  filtered_turbo[columns_to_filter] = turbofan[columns_to_filter].rolling(window=window_size, min_periods=period, axis=0).median()
  return filtered_turbo

In [ ]:
filtered_turbo1=moving_median(turbofan1, 5, 1)
filtered_turbo2=moving_median(turbofan2, 5, 1)
filtered_turbo3=moving_median(turbofan3, 5, 1)
filtered_turbo4=moving_median(turbofan4, 5, 1)

##Data Normalization

In [ ]:
def z_score(turbofan):
  scaler = StandardScaler()
  normalized_turbo = turbofan.copy()
  columns_to_normalize = turbofan.columns.drop('RUL')
  normalized_turbo[columns_to_normalize] = scaler.fit_transform(turbofan[columns_to_normalize])
  return normalized_turbo

In [ ]:
normalized_turbo1=z_score(turbofan1)
normalized_turbo2=z_score(turbofan2)
normalized_turbo3=z_score(turbofan3)
normalized_turbo4=z_score(turbofan4)

In [ ]:
normalized_turbo1

#Implementing the performance measures

In [ ]:
#the asymmetric scoring function  - MASCHLER !!!
def PHME_score(y_true, y_pred):
    dif=y_pred-y_true
    if dif>=0:
      score= math.exp(dif /10)-1
    else:
      score= math.exp(-dif /13)-1
    return score

In [ ]:
print("Score for an overestimation: ", PHME_score(112, 192))
print("Score for an underestimation: ", PHME_score(192, 112))

Score for an overestimation:  2979.9579870417283
Score for an underestimation:  469.5236172558757
